In [3]:
# This script shows you how to make a submission using a few	1	# This script shows you how to make a submission using a few
# useful Python libraries.	2	# useful Python libraries.
# It gets a public leaderboard score of 0.76077.	3	# It gets a public leaderboard score of 0.76077.
# Maybe you can tweak it and do better...?	4	# Maybe you can tweak it and do better...?
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
# Load the data
train_df = pd.read_csv('train.csv', header=0)

test_df = pd.read_csv('test.csv', header=0)

# We'll impute missing values using the median for numeric columns and the most	15	# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.	16	# common value for string columns.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948	17	# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ['Pclass','Sex','Age','Fare','Parch']
nonnumeric_columns = ['Sex']

# Join the features from train and test together before imputing missing values,	31	# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different	32	# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

# XGBoost doesn't (yet) handle categorical features automatically, so we need to change	36	# XGBoost doesn't (yet) handle categorical features automatically, so we need to change
# them to columns of integer values.	37	# them to columns of integer values.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more	38	# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
# details and options	39	# details and options

le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

# Prepare the inputs for the model	44	# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['Survived']

# You can experiment with many other options here, using the same .fit() and .predict()	49	# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org	50	# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost	51	# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = gbm.predict(test_X)

# Kaggle needs the submission to have a certain format;	55	# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv	56	# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.	57	# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],'Survived': predictions })
submission.to_csv("submission.csv", index=False)

In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [11]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],'Survived': predicted })
submission.to_csv("submission_etclf.csv", index=False)

In [13]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import roc_curve

model = BernoulliNB(alpha=1.0, binarize=0.0)
model.fit(train_X, train_y)
pred = np.array(model.predict(test_X))

#evaluation metric : cross-entropy loss.
#log_loss(validation['crime'], predicted) 

In [14]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],'Survived': pred })
submission.to_csv("submission_model.csv", index=False)

In [20]:

from sklearn.ensemble import GradientBoostingClassifier

gdb = GradientBoostingClassifier(learning_rate=0.005, n_estimators=250,
                                max_depth=10, subsample=0.5,
                                max_features=0.5)

gdb = gdb.fit(train_X, train_y)
pre = np.array(gdb.predict(test_X))

submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],'Survived': pre })
submission.to_csv("titanic_gradientboosting.csv", index=False)